## AWS 접근 확인 및 파일 위치 확인

In [3]:
import psycopg2
from psycopg2 import sql
import paramiko
import logging

#  db 연결
def connect_db():
    try:
        conn = psycopg2.connect(
            host = '15.165.7.44',
            database = 'pohang_ai',
            user = 'postgres',
            password = 'yjtech0701!@#'
        )
        
        return conn
    
    except psycopg2.Error as e:
        print(f"Database connection error: {e}")
        return None
    
def load_voice():
    """
    db(mw_audio_analysis)에서 "maf_chk" 표시가 안 된 maf_idx를 찾아서 db(mw_audio_file)의 maf_idx를 찾아
    "maf_save_file_path"를 가져오는 함수
    """
    
    try:
        conn = connect_db()
        cursor = conn.cursor()
        
        # 1. "maf_chk"가 표시되지 않은 maf_idx를 가져옴
        import_idx_query = """
        SELECT maf_idx
        FROM mw_audio_analysis
        WHERE maf_chk = FALSE;
        """
        
        cursor.execute(import_idx_query)
        maf_indices = cursor.fetchall()
        print("\t\t maf_indices:", maf_indices)
        
        if not maf_indices:
            print("No maf_idx found with maf_chk = NULL")
            return None
        
        # maf_idx 추출
        maf_indices = [row[0] for row in maf_indices]
        
        # 2. maf_idx에 해당하는 mw_audio_file 테이블에서 maf_idx와 maf_save_file_path 가져옴
        
        import_file_path_query = """
        SELECT maf_idx, maf_save_file_path 
        FROM mw_audio_file 
        WHERE maf_idx = %s;
        """
        
        voice_data = []
        for maf_idx in maf_indices:
            cursor.execute(import_file_path_query, (maf_idx,))
            result = cursor.fetchall()
            voice_data.extend(result)

        # maf_save_file_path와 maf_idx 리스트 반환
        if voice_data:
            return [{"maf_idx": row[0], "maf_save_file_path": row[1]} for row in voice_data]
        else:
            print("No matching maf_idx found in mw_audio_file")
            return None
        
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

    finally:
        cursor.close()
        conn.close()

def aws_server():

    # AWS EC2 접속 정보
    ec2_host = '43.200.45.232'  # EC2 퍼블릭 IP
    ec2_user = 'ubuntu'  # EC2 사용자 (보통 ec2-user 또는 ubuntu)
    private_key_path = '/home/yjtech/Desktop/LLM/key/bems_aws.pem'

    # SSH 클라이언트 설정
    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    result = {'maf_idx': [], 'file_path': []}  # 결과 저장용 딕셔너리
    
    try:
        # EC2 서버에 접속
        ssh_client.connect(ec2_host, username=ec2_user, key_filename=private_key_path)

        # 데이터베이스에서 반환된 파일 경로 가져오기
        file_data = load_voice()  # 데이터베이스 함수 호출

        if not file_data:
            print("No file paths found from the database.")
            return None

        for file_info in file_data:
            maf_idx = file_info['maf_idx']
            file_path = file_info['maf_save_file_path']

            # print(f"Checking file for maf_idx: {maf_idx}, path: {file_path}")

            # EC2에서 파일 존재 여부 확인
            command = f'ls {file_path}'
            stdin, stdout, stderr = ssh_client.exec_command(command)

            output = stdout.read().decode().strip()
            error = stderr.read().decode().strip()

            if output:  # 파일이 있는 경우
                result['maf_idx'].append(maf_idx)
                result['file_path'].append(file_path)
            elif error:  # 오류가 있는 경우
                print(f"Error for maf_idx {maf_idx}: {error}")
            else:  # 파일이 없는 경우
                print(f"No file found for maf_idx {maf_idx} at {file_path}")

    except Exception as e:
        print(f"Error occurred during AWS server interaction: {e}")
    finally:
        ssh_client.close()
        
    return result

In [8]:
if __name__ == "__main__":
    file_path_check = aws_server()

file_path_check

		 maf_indices: [(48,), (49,), (50,), (51,), (52,), (21,), (40,)]


{'maf_idx': [48, 49, 50, 51, 52, 21, 40],
 'file_path': ['/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-12-16/34102162-eede-475b-b32f-ce6f073d5cc5_test_voice_smell.mp3',
  '/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-12-16/eb9cc347-f910-4679-8d74-896c26a33bcd_test_voice_smell.mp3',
  '/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-12-16/1ff07826-740c-41b2-b542-5ed0503c5703_test_voice_smell.mp3',
  '/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-12-16/06766df2-aa8b-4736-99b8-15ff22c4918f_test_voice_smell.mp3',
  '/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-12-17/3b984625-b6be-485d-b0e6-599beed11b04_test_voice_smell.mp3',
  '/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-11-29/005beadc-4804-4d1e-a300-41fe138d14d4_sample_audio.aiff',
  '/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-12-13/b3683c15-c1e6-4d17-a2e5-b10b8158bfa7_test_voice_smell.mp3']}

In [1]:
import os
import uuid
from pathlib import Path
import paramiko
from scp import SCPClient
import json
import logging

# 로그 설정
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# AWS 서버 정보
AWS_SERVER_HOST = "43.200.45.232"  # AWS 서버 IP
AWS_SERVER_USER = "ubuntu"  # AWS 사용자명
AWS_KEY_PATH = "/home/yjtech/Desktop/LLM/all_llm_pro/key/bems_aws.pem"

# Mac 서버 정보
MAC_SERVER_HOST = "61.37.153.212"  # 서버 IP 주소
MAC_SERVER_USER = "yjtech_mac_machine"          # Mac 서버 사용자명
MAC_SERVER_KEY = "/home/yjtech/Desktop/LLM/all_llm_pro/key/id_rsa"  # Mac 서버 SSH 키 경로

REMOTE_SCRIPT_PATH = "/Users/yjtech_mac_machine/AI/LLM/code/voice_to_text.py"  # Mac 서버의 스크립트 위치
REMOTE_TEMP_DIR = "/Users/yjtech_mac_machine/AI/LLM/data"  # 서버 임시 디렉토리 (파일 저장 없이 작업 수행)

LOCAL_TEMP_DIR = "/home/yjtech/Desktop/LLM/all_llm_pro/aws_file"

def fetch_file_from_aws(aws_file_path, local_temp_dir):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        logger.info("Connecting to AWS...")
        private_key = paramiko.RSAKey.from_private_key_file(AWS_KEY_PATH)
        ssh.connect(hostname=AWS_SERVER_HOST, username=AWS_SERVER_USER, pkey=private_key)
        
        # POSIX 경로 사용
        local_temp_path = Path(local_temp_dir) / Path(aws_file_path).name

        # print(f"Downloading file to: {local_temp_path}")
        with SCPClient(ssh.get_transport()) as scp:
            scp.get(aws_file_path, str(local_temp_path))

        logger.info("Connected to AWS server successfully!")
        # print("File downloaded successfully.")
        return local_temp_path
    finally:
        ssh.close()
        
# 로컬 파일 삭제 함수
def delete_local_file(file_path):
    try:
        if Path(file_path).exists():
            Path(file_path).unlink()
            # print(f"Local file deleted: {file_path}")
        else:
            print(f"File not found: {file_path}")
    except Exception as e:
        logger.error(f"Error deleting local file: {e}")


# Mac에서 파일 실행 및 처리 후 로컬 파일 삭제
def execute_on_mac(maf_idx, file_path):
    ssh = paramiko.SSHClient()
    try:
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        private_key = paramiko.RSAKey.from_private_key_file(MAC_SERVER_KEY)

        logger.info("Connecting to Mac server...")
        ssh.connect(hostname=MAC_SERVER_HOST, username=MAC_SERVER_USER, pkey=private_key)
        logger.info("Connected to Mac server successfully!")

        temp_file_name = Path(file_path).name
        remote_temp_path = f"{REMOTE_TEMP_DIR}/{temp_file_name}"

        # 파일 업로드
        with SCPClient(ssh.get_transport()) as scp:
            scp.put(file_path, remote_temp_path)

        command = (
            f"export PATH=/opt/homebrew/bin:$PATH && "
            f"/opt/anaconda3/envs/venv-keyword/bin/python {REMOTE_SCRIPT_PATH} "
            f"--maf_idx {maf_idx} --file_path {remote_temp_path}"
        )

        # 명령 실행
        stdin, stdout, stderr = ssh.exec_command(command)
        stdout.channel.recv_exit_status()  # Wait for 명령이 완료되기를 기다림

        output = stdout.read().decode().strip()
        error = stderr.read().decode().strip()

        if "Error" in error or "Exception" in error:
            logger.warning(f"Error during script execution: {error}")
            raise Exception(f"Error during script execution: {error}")
        
        # JSON 형식으로 변환
        try:
            result_dict = json.loads(output.split("\n")[-1])  # 마지막 줄에서 JSON 추출
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding JSON: {e}")
            raise Exception("Failed to parse output as JSON")

        return result_dict

    except Exception as e:
        logger.error(f"Error on Mac server: {e}")
        return {'maf_idx': maf_idx, 'voice_to_text': 'Error occurred'}
    
    finally:
        # 로컬 파일 삭제
        delete_local_file(file_path)

        # 원격 파일 삭제
        try:
            # logger.info(f"Deleting remote file: {remote_temp_path}")
            ssh.exec_command(f"rm -f {remote_temp_path}")
        except Exception as e:
            logger.warning(f"Error deleting remote file: {e}")
        finally:
            ssh.close()


In [2]:
import re

def text_clean(maf_idx, voice_to_text_res):
    '''
    텍스트에서 기본적인 전처리 작업을 실행하는 함수
    '''
    result = {
        'maf_idx': maf_idx, 
        'voice_to_text': voice_to_text_res['voice_to_text'], 
        'cleaned_text': None
    }
    
    text = voice_to_text_res['voice_to_text']
    
    if not isinstance(text, str):
        text = str(text)  # 문자열이 아닌 경우 변환
        
    # 한글 자음, 모음 제거
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern, '', text)
    
    # 특수기호 제거: 점(.), 물음표(?), 쉼표(,)는 제외
    pattern = r'(?<!\d)-|\b(?!\d+-\d)\d+\b|[^\w\s.,?\-]'
    text = re.sub(pattern, '', text)

    # 쉼표가 2개 이상 반복될 경우 하나로 축소
    pattern = r',,{1,}'
    text = re.sub(pattern, ',', text)

    # 중복 공백 제거
    pattern = '\s{2, }'
    text = re.sub(pattern, ' ', text).strip()

    # 줄 바꿈 제거
    pattern = '\n'
    text = re.sub(pattern, '', text)

    # `.이 2개 이상` 제거하는 패턴
    pattern = r'\.{2,}'  # 점(.)이 2개 이상인 경우 제거
    text = re.sub(pattern, '', text)
    
    result['cleaned_text'] = text

    return result


In [3]:
import requests
import json
import re

def ensure_list(value):
    """
    입력값이 리스트가 아니면 리스트로 변환하고, 입력값이 None이면 빈 리스트를 반환합니다.
    """
    if value is None:
        return []
    return [value] if not isinstance(value, list) else value

def spacing_and_spelling(maf_idx, cleaned_text_res):
    """
    Ollama API를 호출하여 문장에서 띄어쓰기 및 맞춤법 교정을 하는 함수
    """
    result = {
        'maf_idx': maf_idx, 
        'voice_to_text': cleaned_text_res['voice_to_text'], 
        'spacing_and_spelling_res': None
    }
    
    # Ollama API URL
    url = "http://61.37.153.212:11434/api/generate"
    
    # Request payload
    payload = {
        "model": "gemma2",
        "prompt": f"""
        너는 한국어 맞춤법 검사 도우미야. 주어진 문장을 분석하여 맞춤법과 띄어쓰기를 수정해주세요.
        설명은 괜찮고 수정한 문장만 보내주세요.

        문장: "{cleaned_text_res['cleaned_text']}"

        형식:
        "맞춤법 및 띄어쓰기 수정 결과": "맞춤법과 띄어쓰기를 수정한 문장"
        """
    }

    try:
        # API call
        response = requests.post(url, json=payload, timeout=30)
        
        if response.status_code != 200:
            result['spacing_and_spelling_res'] = {
                'error': f"API Error {response.status_code}: {response.text}"
            }
            return result

        full_response = ""
        for line in response.iter_lines(decode_unicode=True):
            if line:
                try:
                    data = json.loads(line)
                    full_response += data.get("response", "")
                except json.JSONDecodeError:
                    continue
        
        # JSON 추출 정규 표현식
        json_match = re.search(r'(?:"맞춤법 및 띄어쓰기 수정 결과": ".*?")', full_response, re.DOTALL)
        if json_match:
            corrected_text = json_match.group(0)
            result['spacing_and_spelling_res'] = corrected_text.split(': ')[1].strip('"')
            
        else:
            result['spacing_and_spelling_res'] = {
                'error': 'No matching corrected text found',
                'raw_response': full_response
            }
        
        return result
    
    except Exception as e:
        result['spacing_and_spelling_res'] = {
            'error': str(e)
        }
        return result


In [4]:
import requests
import json
import re

def ensure_list(value):
    """
    입력값이 리스트가 아니면 리스트로 변환하고, 입력값이 None이면 빈 리스트를 반환합니다.
    """
    if value is None:
        return []
    return [value] if not isinstance(value, list) else value

def validate_keywords(original_text, extracted_keywords):
    """
    원문에 포함된 키워드인지 검증합니다.
    모든 키워드(time, location, smell_type, smell_intensity)에 대해 원문에서 명시적으로 존재하는 경우만 반환합니다.
    """
    validated_keywords = {}
    for key, values in extracted_keywords.items():
        if isinstance(values, list):
            # 키워드가 리스트일 경우: 원문에 존재하는 단어만 필터링
            validated_keywords[key] = [val for val in values if val in original_text]
        elif isinstance(values, str):
            # 키워드가 문자열일 경우: 원문에 포함된 경우에만 유지
            validated_keywords[key] = values if values in original_text else None
    return validated_keywords


def extract_information(maf_idx, spacing_spelling_res):
    """
    Ollama API를 호출하여 문장에서 시간, 장소, 냄새 종류, 냄새 장소, 냄새 강도 등을 추출합니다.
    """
    result = {
        'maf_idx': maf_idx, 
        'voice_to_text': spacing_spelling_res['voice_to_text'], 
        'keywords': None
    }
    
    # Ollama API URL
    url = "http://61.37.153.212:11434/api/generate"
    
    # Request payload
    payload = {
        "model": "llama3:70b",
        "temperature": 0.0,  # 완전히 결정적으로 설정
        "top_p": 1.0,       # 확률 분포 전체 사용
        "seed": 42,  # seed 값을 추가하여 결정적 결과를 강제
        "prompt": f"""
            너는 민원 데이터를 사용해 중요한 키워드를 추출하는거야
            문장에서 시간, 장소, 냄새 종류, 냄새 강도를 JSON 형식으로 추출해주세요.
            [문장]: "{spacing_spelling_res['spacing_and_spelling_res']}"

            형식:
            {{
                "time": ["시간표현1", "시간표현2"],
                "location": ["장소"],
                "smell_type": ["냄새종류1", "냄새종류2"],
                "smell_intensity": ["강도표현1", "강도표현2"]
            }}

            규칙:
            1. [시간 추출 규칙]:
            - 문장에서 직접적으로 언급된 시간 표현만 포함.
            - 절대적 시간: "오전 9시", "저녁 7시"
            - 상대적 시간: "퇴근", "아침", "저녁"
            - 문장 내에 복합적으로 표현된 시간도 모두 포함.
            - 추론하지 말 것.
            2. [장소 추출 규칙]:
            - 구체적인 위치나 거주지 정보만 포함.
            - 아파트명, 동, 호수 등 상세 정보 포함.
            - 문장 내 민원인의  위치를 추출
            - 예를 들어 힐스테이트 포항, 아이파크, 오천 힐스테이트 등
            3. [냄새 종류 추출 규칙]:
            - 냄새 종류를 추출
            - 냄새 종류에 대한 발생 위치가 포함되어 있다면 같이 추출
            - 문장 내 냄새 종류와 발생 위치를 추출
            4. [냄새 강도 추출 규칙]:
            - 강도를 나타내는 형용사 또는 부사 표현 포함.
            - 문장 내 냄새 강도를 나타내는 표현을추출
            """
    }

    try:
        # API call
        response = requests.post(url, json=payload, timeout=60)
        response.raise_for_status()  # HTTP 오류 상태 코드 확인
        
        # API 응답 처리
        full_response = ""
        for line in response.iter_lines(decode_unicode=True):
            if line:
                data = json.loads(line)
                full_response += data.get("response", "")
        
        # JSON 포맷을 추출하는 정규 표현식
        json_match = re.search(r'```json\n(.*?)\n```', full_response, re.DOTALL)
        if json_match:
            try:
                # JSON 문자열을 파싱하여 키워드 추출
                keywords = json.loads(json_match.group(1))
                validated_keywords = validate_keywords(spacing_spelling_res['spacing_and_spelling_res'], {
                    'time': ensure_list(keywords.get('time', [])),
                    'location': ensure_list(keywords.get('location', ''))[0],
                    'smell_type': ensure_list(keywords.get('smell_type', [])),
                    'smell_intensity': ensure_list(keywords.get('smell_intensity', []))
                })
                result['keywords'] = validated_keywords
            except json.JSONDecodeError:
                result['keywords'] = {
                    'error': 'Failed to parse JSON',
                    'raw_response': full_response
                }
        else:
            # 만약 JSON이 백틱(```)으로 감싸져 있지 않다면 일반적인 JSON 객체로 시도
            json_match = re.search(r'\{[^}]+\}', full_response)
            if json_match:
                try:
                    keywords = json.loads(json_match.group(0))
                    
                    validated_keywords = validate_keywords(spacing_spelling_res['spacing_and_spelling_res'], {
                        'time': ensure_list(keywords.get('time', [])),
                        'location': ensure_list(keywords.get('location', ''))[0],
                        'smell_type': ensure_list(keywords.get('smell_type', [])),
                        'smell_intensity': ensure_list(keywords.get('smell_intensity', []))
                    })
                    result['keywords'] = validated_keywords
                except json.JSONDecodeError:
                    result['keywords'] = {
                        'error': 'Failed to parse JSON',
                        'raw_response': full_response
                    }
            else:
                result['keywords'] = {
                    'error': 'No JSON found',
                    'raw_response': full_response
                }
        return result
    

    except Exception as e:
        result['keywords'] = {
            'error': str(e)
        }
        return result

In [5]:
if __name__ == "__main__":
    # AWS에서 파일 가져오기
    aws_file_path = "/home/ubuntu/pohang_ai/was/file/uploads/minwon/2024-12-06/sample_text2_audio.aiff"
    local_temp_dir = "/home/yjtech/Desktop/LLM/all_llm_pro/aws_file"
    local_file_path = fetch_file_from_aws(aws_file_path, local_temp_dir)

    # Mac 서버에서 작업 실행
    maf_idx = 1
    result_text = execute_on_mac(maf_idx, local_file_path)
    result_cleand = text_clean(maf_idx, result_text)
    print(result_cleand)
    result_spelling = spacing_and_spelling(maf_idx, result_cleand)
    print(result_spelling)
    result = extract_information(maf_idx, result_spelling)
    print(result)

2024-12-20 14:57:38,163 - INFO - Connecting to AWS...
2024-12-20 14:57:38,237 - INFO - Connected (version 2.0, client OpenSSH_8.9p1)
2024-12-20 14:57:38,365 - INFO - Authentication (publickey) successful!
2024-12-20 14:57:38,955 - INFO - Connected to AWS server successfully!
2024-12-20 14:57:39,125 - INFO - Connecting to Mac server...
2024-12-20 14:57:39,168 - INFO - Connected (version 2.0, client OpenSSH_9.7)
2024-12-20 14:57:39,216 - INFO - Authentication (publickey) successful!
2024-12-20 14:57:39,217 - INFO - Connected to Mac server successfully!


{'maf_idx': 1, 'voice_to_text': ' 수고 많으십니다. 오천 힐스테이트 110동, 1005호에 사는 홍길동입니다.  퇴근하고 집에 와서 문을 열었는데 화학약품 냄새가 너무 많이 납니다. 조치 좀 취해주세요.', 'cleaned_text': '수고 많으십니다. 오천 힐스테이트 110동, 1005호에 사는 홍길동입니다.  퇴근하고 집에 와서 문을 열었는데 화학약품 냄새가 너무 많이 납니다. 조치 좀 취해주세요.'}
{'maf_idx': 1, 'voice_to_text': ' 수고 많으십니다. 오천 힐스테이트 110동, 1005호에 사는 홍길동입니다.  퇴근하고 집에 와서 문을 열었는데 화학약품 냄새가 너무 많이 납니다. 조치 좀 취해주세요.', 'spacing_and_spelling_res': '수고 많으십니다. 오천힐스테이트 110동, 1005호에 사는 홍길동입니다. 퇴근하고 집에 와서 문을 열었는데 화학 약품 냄새가 너무 많이 나요. 조치 좀 취해 주세요.'}
{'maf_idx': 1, 'voice_to_text': ' 수고 많으십니다. 오천 힐스테이트 110동, 1005호에 사는 홍길동입니다.  퇴근하고 집에 와서 문을 열었는데 화학약품 냄새가 너무 많이 납니다. 조치 좀 취해주세요.', 'keywords': {'time': ['퇴근'], 'location': '오천힐스테이트 110동', 'smell_type': ['화학 약품 냄새'], 'smell_intensity': ['너무 많이']}}
